## CI/CD Flow in MLOps (Simplified)

1. Developer pushes new model code to GitHub
2. CI/CD pipeline is triggered (GitHub Actions)
3. Pipeline stages:
   a. Lint & test Python code
   b. Run training script → generate model.pkl
   c. Register model in MLflow or S3
   d. Build Docker image with model & API
   e. Push Docker image to registry (e.g., DockerHub)
   f. Deploy using Helm/K8s manifest to Kubernetes cluster
4. Monitor logs, performance, and drift


### Example: CI/CD Pipeline for ML using GitHub Actions

In [ ]:
.
├── train.py
├── model/
│   └── model.pkl
├── app/
│   └── main.py  # FastAPI app
├── Dockerfile
├── requirements.txt
├── .github/
│   └── workflows/
│       └── ml_pipeline.yml


ml_pipeline.yml (Github Actions)

In [ ]:
name: ML CI/CD Pipeline

on:
  push:
    branches: [main]

jobs:
  build-and-deploy:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout Code
      uses: actions/checkout@v2

    - name: Set up Python
      uses: actions/setup-python@v2
      with:
        python-version: 3.9

    - name: Install Dependencies
      run: |
        pip install -r requirements.txt

    - name: Run Unit Tests
      run: |
        pytest tests/

    - name: Train Model
      run: |
        python train.py

    - name: Build Docker Image
      run: |
        docker build -t yourdockerhubuser/ml-fastapi:latest .

    - name: Push Docker Image
      run: |
        echo "${{ secrets.DOCKER_PASSWORD }}" | docker login -u "${{ secrets.DOCKER_USERNAME }}" --password-stdin
        docker push yourdockerhubuser/ml-fastapi:latest

    - name: Deploy to Kubernetes
      run: |
        kubectl apply -f k8s/deployment.yaml




### CI/CD pipeline using GitHub Actions to automate the process of:

Model training

Packaging and serving via FastAPI

Docker image build & push

Kubernetes deployment

####  Project Structure

In [ ]:
mlops-pipeline/
│
├── .github/
│   └── workflows/
│       └── ci-cd.yml        <-- GitHub Actions workflow
│
├── app/
│   ├── model/
│   │   ├── train.py         <-- ML training script
│   │   └── model.pkl        <-- Trained model (generated)
│   │
│   └── main.py              <-- FastAPI app to serve model
│
├── Dockerfile
├── k8s/
│   ├── deployment.yaml
│   └── service.yaml
├── requirements.txt
└── README.md


#### Step 1: ML Training Script (train.py)

In [ ]:
# app/model/train.py
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)
model = LogisticRegression()
model.fit(X, y)

# Save model
with open("app/model/model.pkl", "wb") as f:
    pickle.dump(model, f)


#### Step 2: FastAPI App (main.py)

In [ ]:
# app/main.py
from fastapi import FastAPI
import pickle
from pydantic import BaseModel

# Load model
with open("app/model/model.pkl", "rb") as f:
    model = pickle.load(f)

app = FastAPI()

class InputData(BaseModel):
    features: list

@app.post("/predict")
def predict(data: InputData):
    pred = model.predict([data.features])
    return {"prediction": pred.tolist()}


#### Step 3: Dockerfile

In [ ]:
# Dockerfile
FROM python:3.10-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install -r requirements.txt

COPY app/ ./app/

CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "80"]


#### Step 4: Kubernetes Deployment YAML

In [ ]:
# k8s/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: ml-model
spec:
  replicas: 1
  selector:
    matchLabels:
      app: ml-model
  template:
    metadata:
      labels:
        app: ml-model
    spec:
      containers:
        - name: ml-container
          image: your-dockerhub-username/ml-model:latest
          ports:
            - containerPort: 80



# k8s/service.yaml
apiVersion: v1
kind: Service
metadata:
  name: ml-service
spec:
  selector:
    app: ml-model
  ports:
    - protocol: TCP
      port: 80
      targetPort: 80
  type: LoadBalancer


#### Step 5: GitHub Actions Workflow

In [ ]:
# .github/workflows/ci-cd.yml
name: MLOps CI/CD

on:
  push:
    branches:
      - main

jobs:
  build-and-deploy:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout Repo
      uses: actions/checkout@v3

    - name: Set up Python
      uses: actions/setup-python@v4
      with:
        python-version: '3.10'

    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt

    - name: Train Model
      run: |
        python app/model/train.py

    - name: Build Docker image
      run: |
        docker build -t your-dockerhub-username/ml-model:latest .

    - name: Login to DockerHub
      uses: docker/login-action@v2
      with:
        username: ${{ secrets.DOCKER_USERNAME }}
        password: ${{ secrets.DOCKER_PASSWORD }}

    - name: Push Docker Image
      run: |
        docker push your-dockerhub-username/ml-model:latest

    - name: Deploy to K8s
      uses: azure/k8s-deploy@v4
      with:
        manifests: |
          k8s/deployment.yaml
          k8s/service.yaml
        images: |
          your-dockerhub-username/ml-model:latest
        namespace: default
        kubeconfig: ${{ secrets.KUBECONFIG }}


### CI/CD pipeline for a Machine Learning project using the following tools:

Jenkins – CI/CD orchestration

MLflow – Experiment tracking

Docker – Containerizing model + FastAPI

Helm – Kubernetes deployment management

### Directory Structure (for reference)

In [ ]:
mlops-project/
├── jenkins/
│   └── Jenkinsfile
├── mlflow/
│   └── train_model.py
├── app/
│   ├── main.py
│   ├── requirements.txt
│   └── Dockerfile
├── helm-chart/
│   └── fastapi-ml-chart/
│       ├── Chart.yaml
│       ├── values.yaml
│       └── templates/
│           └── deployment.yaml
├── .dockerignore
└── .gitignore


#### 1. Jenkinsfile

In [ ]:
pipeline {
    agent any

    environment {
        DOCKER_IMAGE = 'yourdockerhubuser/iris-api:latest'
        DOCKER_CREDENTIALS_ID = 'dockerhub-creds'
    }

    stages {
        stage('Train and Log with MLflow') {
            steps {
                sh 'python3 mlflow/train_model.py'
            }
        }

        stage('Build Docker Image') {
            steps {
                sh 'docker build -t $DOCKER_IMAGE ./app'
            }
        }

        stage('Push Docker Image') {
            steps {
                withCredentials([usernamePassword(credentialsId: "${DOCKER_CREDENTIALS_ID}", usernameVariable: 'DOCKER_USER', passwordVariable: 'DOCKER_PASS')]) {
                    sh 'echo "$DOCKER_PASS" | docker login -u "$DOCKER_USER" --password-stdin'
                    sh 'docker push $DOCKER_IMAGE'
                }
            }
        }

        stage('Deploy to Kubernetes with Helm') {
            steps {
                sh 'helm upgrade --install iris-api ./helm-chart/fastapi-ml-chart --set image.repository=$DOCKER_IMAGE'
            }
        }
    }
}


#### 2. MLflow Training Script (mlflow/train_model.py)

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

mlflow.set_tracking_uri("http://your-mlflow-server:5000")  # Or localhost
mlflow.set_experiment("iris-exp")

with mlflow.start_run():
    iris = load_iris()
    X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2)

    model = LogisticRegression()
    model.fit(X_train, y_train)
    acc = accuracy_score(y_test, model.predict(X_test))

    mlflow.log_metric("accuracy", acc)
    mlflow.sklearn.log_model(model, "model")


####  3. FastAPI App (app/main.py)

In [ ]:
from fastapi import FastAPI
import mlflow.sklearn
import pandas as pd

app = FastAPI()
model = mlflow.sklearn.load_model("model")  # Or local path or URI

@app.get("/")
def read_root():
    return {"message": "Model is ready"}

@app.post("/predict")
def predict(features: list):
    df = pd.DataFrame([features])
    pred = model.predict(df)
    return {"prediction": int(pred[0])}


#### 4. Dockerfile (app/Dockerfile)

In [ ]:
FROM python:3.9-slim

WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "80"]


#### requirements.txt

In [ ]:
fastapi
uvicorn
mlflow
scikit-learn
pandas


#### 5. Helm Chart (helm-chart/fastapi-ml-chart/values.yaml)

In [ ]:
replicaCount: 1

image:
  repository: yourdockerhubuser/iris-api
  pullPolicy: Always
  tag: latest

service:
  type: ClusterIP
  port: 80

resources: {}
